# Uploading CSV from previous EDA

In [1]:
import pandas as pd 
from tensorflow import feature_column as fc
import tensorflow as tf
path = "Preprocessing.csv"
df = pd.read_csv(path)


# Preprocessing

In [2]:
df.drop(df[df['gender']=='Other'].index, inplace=True)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,28.893237,never smoked,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.000000,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.600000,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.600000,formerly smoked,0


In [3]:
cat_columns = []
num_columns = []
target = ['stroke']
for i in df:
  if i != 'id' and i != 'stroke':
    if df[i].dtypes == 'object':
        cat_columns.append(i)
    elif df[i].dtypes != 'object':
        num_columns.append(i)
          
cat_columns, num_columns, target

(['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'],
 ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi'],
 ['stroke'])

In [4]:
columns = df.columns
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [5]:
def one_hat_encoding(feature_name, vocab):
    return fc.indicator_column(fc.categorical_column_with_vocabulary_list(feature_name, vocab))

feature_columns = []
for feat_name in cat_columns :
    vocabulary = X_train[feat_name].unique()
    feature_columns.append(one_hat_encoding(feat_name, vocabulary))
    
for feat_name in num_columns:
    feature_columns.append(fc.numeric_column(feat_name, dtype=tf.float32))
    
example_batch = next(iter(X_train))[0]
def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())
    

tf.keras.layers.DenseFeatures(feature_columns)(dict(X_train.head(1))), X_train.columns, feature_columns

(<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
 array([[ 1.  ,  0.  , 79.  , 72.04, 23.6 ,  1.  ,  0.  ,  1.  ,  0.  ,
          0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
          0.  ,  0.  ]], dtype=float32)>,
 Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
        'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
        'smoking_status'],
       dtype='object'),
 [IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
  IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='ever_married', vocabulary_list=('Yes', 'No'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
  IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='work_type', vocabulary_list=('Self-employed', 'Private', 'children', 'Govt_job', 'Never_worked'), dtype=tf.string, default_value=-1, num_oov_bucket

In [6]:
def make_input_fn(X, Y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((dict(X),Y))
        if shuffle :
            dataset = dataset.shuffle(32)
        dataset = dataset.repeat(n_epochs)
        dataset = dataset.batch(32)
        return dataset
    return input_fn

train_input_fn = make_input_fn(X_train, Y_train)
eval_input_fn = make_input_fn(X_test, Y_test, shuffle=False, n_epochs=1)

In [7]:
tree = tf.estimator.BoostedTreesClassifier(feature_columns, n_batches_per_layer=1)
tree.train(train_input_fn, max_steps=100)
result = tree.evaluate(eval_input_fn)
print(pd.Series(result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ASUS\\AppData\\Local\\Temp\\tmpkp179d7y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
The old _FeatureColu

Exception ignored in: <function CapturableResource.__del__ at 0x0000022DD01891F0>
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 269, in __del__
    with self._destruction_context():
AttributeError: 'TreeEnsemble' object has no attribute '_destruction_context'


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ASUS\AppData\Local\Temp\tmpkp179d7y\model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0

Exception ignored in: <function CapturableResource.__del__ at 0x0000022DD01891F0>
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 269, in __del__
    with self._destruction_context():
AttributeError: 'TreeEnsemble' object has no attribute '_destruction_context'


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-23T19:58:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ASUS\AppData\Local\Temp\tmpkp179d7y\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.03723s
INFO:tensorflow:Finished evaluation at 2021-05-23-19:58:05
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.47358122, accuracy_baseline = 0.9412916, auc = 0.57191616, auc_precision_recall = 0.3875693, average_loss = 59776136.0, global_step = 100, label/mean = 0.058708414, loss = 59932130.0, precision = 0.073214285, prediction/mean = 0.5479452, recall = 0.68333334
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: C:\Use

In [35]:
def make_input_fn1(X):
    def input_fn():
        dataset = tf.data.Dataset.from_tensors((dict(X))).batch(256)
        return dataset
    return input_fn

columns = [x for x in df.columns if x != 'stroke']
d = {'gender':['Male'], 
    'age':[25], 
    'hypertension':[0], 
    'heart_disease':[0],
    'ever_married':['No'],
    'work_type':['Never_worked'], 
    'Residence_type':['Urban'],
    'avg_glucose_level':[150],
    'bmi':[21.54], 
    'smoking_status':['never smoked']}
df_test = pd.DataFrame(data=d)
df_test
df_test = make_input_fn1(df_test)
pred_dicts1 = list(tree.predict(df_test))
pred = pred_dicts1[0]['probabilities'][1] > 0.5
pred


INFO:tensorflow:Calling model_fn.


Exception ignored in: <function CapturableResource.__del__ at 0x0000022DD01891F0>
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 269, in __del__
    with self._destruction_context():
AttributeError: 'TreeEnsemble' object has no attribute '_destruction_context'


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ASUS\AppData\Local\Temp\tmpkp179d7y\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


False

In [9]:
linear = tf.estimator.LinearClassifier(feature_columns)
linear.train(train_input_fn, max_steps=100)
result = linear.evaluate(eval_input_fn)
print(pd.Series(result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ASUS\\AppData\\Local\\Temp\\tmp5qrc9gsw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


c:\users\asus\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ASUS\AppData\Local\Temp\tmp5qrc9gsw\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\ASUS\AppData\Local\Temp\tmp5qrc9gsw\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 0.0016107616.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
IN

In [39]:
d = {'gender':['Male'], 
    'age':[25], 
    'hypertension':[0], 
    'heart_disease':[0],
    'ever_married':['No'],
    'work_type':['Private'], 
    'Residence_type':['Urban'],
    'avg_glucose_level':[145],
    'bmi':[26.79], 
    'smoking_status':['never smoked']}
df_test = pd.DataFrame(data=d)
df_test
df_test = make_input_fn1(df_test)
pred_dicts1 = list(linear.predict(df_test))
pred = pred_dicts1[0]['probabilities'][1] > 0.5
pred


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ASUS\AppData\Local\Temp\tmp5qrc9gsw\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


False

# Documentasi 23.05.2021

In this Colab, we are doing data preprocessing and making a model. 
For Preprocessing we:

1. Preprocessing data with tensorflow.data

2. Spliting the features and the target 

3. Splitting the dataset into Training set and Test set using random_seed

4. Comparing 2 machine learnig algorithm(Linear Claasifier and Boosted Trees Classifier 

5. Linear Classifier have the best accuracy 94%


For model we using :

1. Linear Classifier have accuracy 94%

2. Boosted Trees Classifier have accuracy 47,3 %